In [1]:
import io

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

import json
# from pdfminer.converter import TextConverter
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfpage import PDFPage

from bs4 import BeautifulSoup
import os
import re
import requests
import string
import numpy as np
import pandas as pd
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

/home/noahdarwich/.pyenv/versions/lewagon/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': 'https://www.al-akhbar.com/Editions',
    },
    )
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
container = soup.find('div', {'class': 'l-content row archive-issues'})
container = container.find('div', id='archive-days-wrap')

In [ ]:
dic = {}

for a_tag in container.find_all('a', class_='day'):
    data_id = a_tag.get('data-id')
    link = a_tag.get('href')
    url = 'https://www.al-akhbar.com' + link + '/PDF'
    data_name = a_tag.get('data-oldtitle')
    date = link[10:].replace('/','-')
    dic[data_id] = [data_name,url,date]

In [ ]:
df = pd.DataFrame(data=dic).T
df.columns = ['issue', 'url', 'date']
df = df[(df['date'] > '2010-01-01') & (df['date'] < '2020-01-01')]

In [ ]:

def get_pdf(url):
    
    retries = Retry(total=5,
                status_forcelist=[429, 500, 502, 503, 504])
    response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': url,
    },
    )
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        container = soup.find('div', {'class': 'content'})
        pdf_link = container.find('a').get('href')
        pdf_link = 'https://www.al-akhbar.com' + pdf_link
        print('tamam')
        return pdf_link
    
    except:
        return None

In [ ]:
mask = (df.pdf_link.isna())
df2 = df.loc[mask]
df2['pdf_link'] = df2['url'].apply(get_pdf)
df2.to_csv('new.csv')

In [ ]:
mask3 = (df2.pdf_link.isna())
df3 = df2.loc[mask3]
df3

In [ ]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3.to_csv('new2.csv')

In [ ]:
df3 = pd.read_csv('new3.csv')
df3

In [ ]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3

In [ ]:
df = pd.read_csv('pdf_df.csv')
df

In [ ]:
df1 = pd.read_csv('new.csv')
df2 = pd.read_csv('new2.csv')
df3 = pd.read_csv('new3.csv')

In [ ]:
df

In [ ]:
df.pdf_link.isna().value_counts()

In [ ]:
full_df = pd.concat([df1,df2,df3], ignore_index=True)
full_df.drop(columns=['Unnamed: 0'], inplace= True)

In [ ]:
full_df.drop_duplicates(inplace=True)

In [ ]:
full_df.pdf_link.isna().value_counts()

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = full_df['pdf_link']

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df['url'].apply(get_pdf)

In [ ]:
df.pdf_link.isna().value_counts()

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df.loc[df['pdf_link'].isna(), 'url'].apply(get_pdf)

In [ ]:
df.to_csv('Full_df.csv')

In [3]:
df = pd.read_csv('Full_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,issue,url,date,pdf_link
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,https://www.al-akhbar.com/PDF_Files/3885/alakh...
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...
...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...


In [ ]:
os.chdir('/home/noahdarwich/code/NoahDarwich/Leb_analysis/Notebooks/')

In [4]:
os.getcwd()

'/home/noahdarwich/code/NoahDarwich/Leb_analysis/Notebooks'

In [17]:
# def save_pdfs(df):
    
#     unsaved = []
#     df = df.dropna()
#     for i, j in df.iterrows():
        
#         name = j['date']+'.pdf'
#         url = j['pdf_link']
        
#         response = requests.get(url)
        
#         try:
#             os.chdir('/mnt/d')
#             with open(name, 'wb') as file:
#                 file.write(response.content)
#                 print('tamam')
#         except:
#             unsaved.append(name)
#             print('No')
#     return unsaved

In [4]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])

In [4]:
test = df.iloc[0:10]
test.iloc[1]['pdf_link'] = None
test

In [15]:
def save_pdfs(df,creds):
    
    service = build('drive', 'v3', credentials=creds)
    df['status'] = None
    
    for _,row in df.iterrows():
        name = row['date'] +'.pdf'
        link = row['pdf_link']
        folder_id = '1BVzI84DDDtbvU3pp1k0yERF5YKIs84R0'
        results = service.files().list(q=f"mimeType != 'application/vnd.google-apps.folder' and trashed = false and name='{name}' and '{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
        files = results.get("files", [])
        if files:
            print(F'file with name {name} already exists in the folder')
        else:
            
            if str(link).startswith('https://'):
                try:
                    response = requests.get(link)
                    file = tempfile.NamedTemporaryFile()
                    file.write(response.content)
                    file.seek(0)
                    
                    file_metadata = {'name': name,'parents':[folder_id]}
                    
                    media = MediaFileUpload(file.name,
                                        mimetype='application/pdf')
                    
                    service.files().create(body=file_metadata, media_body=media, fields='id').execute()
                    row['status'] = 'saved'
                except:
                    row['status'] = 'not_saved'
            else:
                row['status'] = 'not_saved'
    return df

In [5]:
def save_pdfs_(df,creds):
    
    service = build('drive', 'v3', credentials=creds)
    
    for _,row in df.iterrows():
        name = row['date'] +'.pdf'
        link = row['pdf_link']
        folder_id = '1OIT-XMGYtQd-EPNParT8dDVegLih-NZc'
        results = service.files().list(q=f"mimeType != 'application/vnd.google-apps.folder' and trashed = false and name='{name}' and '{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
        files = results.get("files", [])
        if files:
            print(F'file with name {name} already exists in the folder')
        else:
            response = requests.get(link)
            file = tempfile.NamedTemporaryFile()
            file.write(response.content)
            file.seek(0)
            
            file_metadata = {'name': name,'parents':[folder_id]}
            
            media = MediaFileUpload(file.name,
                                mimetype='application/pdf')
            
            service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            print('ok')
            
    return df

In [7]:
df_results = pd.read_csv('df_results.csv', index_col=0)

In [8]:
df = df_results[df_results['status'] == 'corrupted']

df_ = save_pdfs_(df,creds)
df_

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


,issue,url,date,pdf_link,status
583,العدد 3383,https://www.al-akhbar.com/Editions/2018/1/30/PDF,2018-1-30,https://www.al-akhbar.com/PDF_Files/3383/alakh...,corrupted
746,العدد 3202,https://www.al-akhbar.com/Editions/2017/6/16/PDF,2017-6-16,https://www.al-akhbar.com/PDF_Files/3202/alakh...,corrupted
1025,العدد 2904,https://www.al-akhbar.com/Editions/2016/6/7/PDF,2016-6-7,https://www.al-akhbar.com/PDF_Files/2904/alakh...,corrupted
1566,العدد 2361,https://www.al-akhbar.com/Editions/2014/8/6/PDF,2014-8-6,https://www.al-akhbar.com/PDF_Files/1388/alakh...,corrupted
1653,العدد 2300,https://www.al-akhbar.com/Editions/2014/5/22/PDF,2014-5-22,https://www.al-akhbar.com/PDF_Files/1282/alakh...,corrupted
...,...,...,...,...,...
1941,العدد 2009,https://www.al-akhbar.com/Editions/2013/5/22/PDF,2013-5-22,https://www.al-akhbar.com/PDF_Files/1029/alakh...,corrupted
1942,العدد 2010,https://www.al-akhbar.com/Editions/2013/5/23/PDF,2013-5-23,https://www.al-akhbar.com/PDF_Files/1030/alakh...,corrupted
1943,العدد 2011,https://www.al-akhbar.com/Editions/2013/5/24/PDF,2013-5-24,https://www.al-akhbar.com/PDF_Files/1031/alakh...,corrupted
1944,العدد 2012,https://www.al-akhbar.com/Editions/2013/5/25/PDF,2013-5-25,https://www.al-akhbar.com/PDF_Files/1032/alakh...,corrupted


In [80]:
df_results.to_csv('df_results.csv')

In [4]:
df_results = pd.read_csv('df_results.csv', index_col=0)
df_results

,issue,url,date,pdf_link,status
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN
...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved


In [5]:
[x for x in df_results[df_results['status'] == 'not_saved']['date']]

['2019-12-3',
 '2018-2-12',
 '2012-12-4',
 '2012-11-7',
 '2012-9-28',
 '2012-8-2',
 '2012-8-25',
 '2012-6-22',
 '2012-4-6',
 '2012-3-31',
 '2012-1-5',
 '2012-1-5',
 '2011-10-6',
 '2011-10-6',
 '2011-10-16',
 '2011-5-19',
 '2011-4-6',
 '2011-4-6',
 '2011-4-11',
 '2011-4-11',
 '2011-3-11',
 '2011-3-11',
 '2011-3-16',
 '2011-3-16',
 '2011-2-3',
 '2011-2-3',
 '2011-2-4',
 '2011-2-4',
 '2010-12-13',
 '2010-12-15',
 '2010-12-16',
 '2010-12-18',
 '2010-8-10']

In [12]:
dates = df_results.date
dates

0       2019-12-2
1       2019-12-3
2       2019-12-4
3       2019-12-5
4       2019-12-6
          ...    
2923    2010-1-26
2924    2010-1-27
2925    2010-1-28
2926    2010-1-29
2927    2010-1-30
Name: date, Length: 2928, dtype: object

In [18]:
from datetime import datetime, timedelta

def missing_dates(dates):
    dates = [datetime.strptime(d, "%Y-%m-%d") for d in dates]
    start, end = min(dates), max(dates)
    total_days = (end - start).days + 1
    all_dates = [start + timedelta(days=i) for i in range(total_days)]
    return [d.strftime("%Y-%m-%d") for d in all_dates if d not in dates]

missing_dates_ = missing_dates(dates)
for x in missing_dates_:
    print(x)

2010-01-02
2010-01-03
2010-01-07
2010-01-10
2010-01-17
2010-01-24
2010-01-31
2010-02-07
2010-02-10
2010-02-14
2010-02-21
2010-02-27
2010-02-28
2010-03-07
2010-03-14
2010-03-21
2010-03-26
2010-03-28
2010-04-04
2010-04-05
2010-04-11
2010-04-18
2010-04-25
2010-05-02
2010-05-07
2010-05-09
2010-05-16
2010-05-23
2010-05-26
2010-05-30
2010-06-06
2010-06-13
2010-06-20
2010-06-27
2010-07-04
2010-07-10
2010-07-11
2010-07-18
2010-07-25
2010-08-01
2010-08-08
2010-08-15
2010-08-16
2010-08-22
2010-08-29
2010-09-05
2010-09-11
2010-09-12
2010-09-19
2010-09-26
2010-10-03
2010-10-10
2010-10-17
2010-10-24
2010-10-31
2010-11-07
2010-11-14
2010-11-17
2010-11-18
2010-11-21
2010-11-23
2010-11-28
2010-12-05
2010-12-08
2010-12-10
2010-12-11
2010-12-12
2010-12-14
2010-12-17
2010-12-19
2010-12-25
2010-12-26
2011-01-01
2011-01-02
2011-01-07
2011-01-09
2011-01-16
2011-01-23
2011-01-30
2011-02-06
2011-02-10
2011-02-13
2011-02-16
2011-02-20
2011-02-27
2011-03-03
2011-03-04
2011-03-06
2011-03-12
2011-03-13
2011-03-17

In [19]:
for x in missing_dates_:
    if x in dates:
        print(x)

In [83]:
df_notdone = df_results[(df_results['status']!='saved')&(df_results['status']!='not_saved')]
df_notdone

,issue,url,date,pdf_link,status
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN
5,العدد 3927,https://www.al-akhbar.com/Editions/2019/12/7/PDF,2019-12-7,https://www.al-akhbar.com/PDF_Files/3927/alakh...,NaN
...,...,...,...,...,...
542,العدد 3389,https://www.al-akhbar.com/Editions/2018/2/6/PDF,2018-2-6,https://www.al-akhbar.com/PDF_Files/3389/alakh...,NaN
543,العدد 3390,https://www.al-akhbar.com/Editions/2018/2/7/PDF,2018-2-7,https://www.al-akhbar.com/PDF_Files/3390/alakh...,NaN
544,العدد 3391,https://www.al-akhbar.com/Editions/2018/2/8/PDF,2018-2-8,https://www.al-akhbar.com/PDF_Files/3391/alakh...,NaN
545,العدد 3392,https://www.al-akhbar.com/Editions/2018/2/9/PDF,2018-2-9,https://www.al-akhbar.com/PDF_Files/2426/alakh...,NaN


In [90]:
# df_results_2 = save_pdfs(df_notdone,creds)
# df_results_2

In [91]:
# service = build('drive', 'v3', credentials=creds)


# name = df_results.iloc[1]['date'] +'.pdf'
# link = df_results.iloc[1]['pdf_link']
# response = requests.get(link)
# print(response)
    #                 file = tempfile.NamedTemporaryFile()
    #                 file.write(response.content)
    #                 file.seek(0)
                    
    #                 file_metadata = {'name': name,'parents':[folder_id]}
                    
    #                 media = MediaFileUpload(file.name,
    #                                     mimetype='application/pdf')
                    
    #                 service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    #                 row['status'] = 'saved'
    #             except:
    #                 row['status'] = 'not_saved'
    #         else:
    #             row['status'] = 'not_saved'
    # return df

In [95]:
service = build('drive', 'v3', credentials=creds)
folder_id = '1BVzI84DDDtbvU3pp1k0yERF5YKIs84R0'
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
files = results.get("files", [])
next_page_token = results.get("nextPageToken")
while next_page_token:
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
    files.extend(results.get("files", []))
    next_page_token = results.get("nextPageToken")
print(F'Number of files in the folder "AAA" : {len(files)}')

Number of files in the folder "AAA" : 2894


In [101]:
df_results[(df_results['status']=='not_saved')].shape

(33, 5)

# move files to their folders

In [15]:
year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn',}

In [37]:
service = build('drive', 'v3', credentials=creds)
folder_id = year_folder_id_dict['2014']
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
files = results.get("files", [])
next_page_token = results.get("nextPageToken")
while next_page_token:
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
    files.extend(results.get("files", []))
    next_page_token = results.get("nextPageToken")

In [38]:
len(files)

295

In [16]:
for file in files:
    name = file.get('name')
    year = ''.join(x for x in name[:4] if x in string.digits)
    if year:
        year_folder_id = year_folder_id_dict.get(year)
        if year_folder_id:
             file_id = file.get('id')
             service.files().update(fileId=file_id, addParents=year_folder_id, removeParents=folder_id, fields='id, parents').execute()


In [ ]:
# def save_article_to_drive(article_text, file_name):
#     try:
#       # authenticate with Google Drive
#       credentials = Credentials.from_authorized_user_info()
#       service = build('drive', 'v3', credentials=credentials)

#       # create the file on Google Drive
#       file_metadata = {'name': file_name, 'mimeType': 'text/plain'}
#       media = MediaIoBaseUpload(io.StringIO(article_text), mimetype='text/plain',chunksize=1024*1024, resumable=True)
#       file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
#       print(f'File ID: {file.get(/id\)}')
      
#     except HttpError as error:
#         print(f'An error occurred: {error}')
#         file = None
#     return file

In [ ]:
# scrape the website for PDF links

# download each PDF and save it to Google Drive

# text_ls = [] 
# for link in pdf_links:
#     # download the PDF
#     pdf_url = link['href']
    
#     pdf_content = pdf_response.content
#     # extract the text from the PDF
#     text = extract_text_from_pdf(pdf_content)
#     text_ls.append(text)
    
# print(soup)

In [ ]:
# def extract_text_from_pdf(pdf_content):
#   resource_manager = PDFResourceManager()
#   fake_file_handle = io.StringIO()
#   converter = TextConverter(resource_manager, fake_file_handle)
#   page_interpreter = PDFPageInterpreter(resource_manager, converter)

#   with io.BytesIO(pdf_content) as fh:
#     for page in PDFPage.get_pages(fh,
#                                   caching=True,
#                                   check_extractable=True):
#       page_interpreter.process_page(page)

#     text = fake_file_handle.getvalue()

#   # close open handles
#   converter.close()
#   fake_file_handle.close()

#   return text

In [ ]:
# "# search the text for keywords
# "keywords = ['keyword1', 'keyword2', 'keyword3']
# "for keyword in keywords:
# "  if re.search(keyword, text, re.IGNORECASE):
# "    # keyword found, save the article
# "    file_name = f'{keyword}_article.txt'
# "    save_article_to_drive(text, file_name)
# "    break"